In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedShuffleSplit
import warnings

C:\Users\preductor\AppData\Local\conda\conda\envs\py3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
raw_data = pd.read_csv(r'C:\Users\preductor\Documents\MachineLearning\6_course\Сhurn_prediction\week_4\orange_small_churn_train_data.csv',
                  engine='python')

In [3]:
raw_data.head()

,ID,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,...,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230,labels
0,0,NaN,NaN,NaN,NaN,NaN,3052.0,NaN,NaN,NaN,...,vr93T2a,LM8l689qOp,NaN,NaN,fKCe,02N6s8f,xwM2aC7IdeMC0,NaN,NaN,-1
1,1,NaN,NaN,NaN,NaN,NaN,1813.0,7.0,NaN,NaN,...,6hQ9lNX,LM8l689qOp,NaN,ELof,xb3V,RAYp,55YFVY9,mj86,NaN,-1
2,2,NaN,NaN,NaN,NaN,NaN,1953.0,7.0,NaN,NaN,...,catzS2D,LM8l689qOp,NaN,NaN,FSa2,ZI9m,ib5G6X1eUxUn6,mj86,NaN,-1
3,3,NaN,NaN,NaN,NaN,NaN,1533.0,7.0,NaN,NaN,...,e4lqvY0,LM8l689qOp,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN,1
4,4,NaN,NaN,NaN,NaN,NaN,686.0,7.0,NaN,NaN,...,MAz3HNj,LM8l689qOp,NaN,NaN,WqMG,RAYp,F2FyR07IdsN7I,NaN,NaN,-1


In [4]:
print('Процент churn обьектов: {}%'.format(round(raw_data[raw_data.labels==1].shape[0]/raw_data.shape[0]*100,2)))

Процент churn обьектов: 7.44%


### Определим данные с большим количеством NaN

In [5]:
table_of_nan = pd.DataFrame(index = raw_data.columns, columns=['nans'])
for col in raw_data.columns:
    table_of_nan.loc[col,'nans'] = round((raw_data[col].isna().sum())/raw_data[col].shape[0]*100, 2)
table_of_nan.sort_values(by='nans', ascending=False, inplace=True)
table_of_nan.head(3)

,nans
Var52,100
Var141,100
Var55,100


In [6]:
def percent_nans(percentage):    
    print('Признаков, где Nan объектов меньше {}%: {}%'.format(percentage,
                    round(100*table_of_nan[table_of_nan.nans<percentage].shape[0]/table_of_nan.shape[0], 2)))

In [7]:
print('Всего Nan значений в данных {}%'.format(round(table_of_nan.nans.sum()/table_of_nan.shape[0],2)))

Всего Nan значений в данных 69.18%


In [263]:
percent_nans(98)

Признаков, где Nan объектов меньше 98%: 68.53%


### <font color='green'>Теперь я попытаюсь собрать в кучу наброски из прошлых недель и составить удобные функции для дальнейшего комбинирования различных вариантов обучения нашей модели, а затем и их тестирования </font>

### Первым делом создам функцию отсеивающую признаки, в которых не определен заданный процент значений

In [9]:
def filter_attr(percentage):
    temp = raw_data.loc[:,table_of_nan[table_of_nan.nans<percentage].index]
    X, hold_out_X, y, hold_out_y = train_test_split(temp.drop('labels',axis=1),
                                                temp.labels, stratify=temp.labels, test_size=0.2)
    data = X
    data['y'] = y
    hold_out = hold_out_X
    hold_out['y'] = hold_out_y
    return data, hold_out

In [10]:
data, hold_out = filter_attr(75)

### Выделим вещественные и категориальные данные из отфильтрованных данных для дальнейшей обработки

In [11]:
def type_processing(data):
    num_cols = []
    cat_cols = []
    for col in data.drop(columns=['ID','y'], axis=1).columns:
        if (int(str(col)[3:])<190):
            num_cols.append(col)
        else:
            cat_cols.append(col)
    return data.loc[:,num_cols], data.loc[:,cat_cols]

In [12]:
X_num, X_cat = type_processing(data)

### Начну с вещественных данных, к которым применю imputer с заданным типом наполнения Nan значений,  а затем стандартизирую

In [13]:
def num_processing(X_num, strategy):
    imp = Imputer(strategy=strategy, axis=0)
    X_num_imputed = imp.fit_transform(X_num)
    return pd.DataFrame(data=StandardScaler().fit_transform(X_num_imputed), index=X_num.index,
                         columns=X_num.columns)

### К категориальным применю разные подходы

In [255]:
#таблица количества уникальных значений
number_of_unique = pd.DataFrame(index=X_cat.columns, columns=['amount'])
for col in X_cat.columns:
    number_of_unique.loc[col,'amount'] = X_cat.loc[:,col].unique().shape[0]
number_of_unique.sort_values(by='amount', inplace=True)
number_of_unique.head(3)

,amount
Var211,2
Var201,3
Var208,3


#### В первом случае просто выкину все признаки у которых уникальных значений меньше 10 и применю OneHotEncoding к оставшимся

In [15]:
def cat_1(X_cat):
    try:
        return pd.get_dummies(X_cat.loc[:, number_of_unique[number_of_unique.amount<10].index])        
    except:
        print('Не осталось категориальных признаков с кол-вом значений меньше 10')

#### Во втором - оставлю еще и те признаки, уникальных значений в которых меньше 50 и применю к ним LabelEncoder

In [250]:
def cat_2(X_cat):
    try:
        X_cat_less_10 = pd.get_dummies(X_cat.loc[:, number_of_unique[number_of_unique.amount<10].index])
        X_cat_labelen = X_cat.loc[:, number_of_unique[50>number_of_unique.amount]
                              [number_of_unique.amount>=10].index].astype('str') \
                                    .apply(LabelEncoder().fit_transform)
        return pd.concat([X_cat_less_10, X_cat_labelen], axis=1)
    except:
        print('Не осталось категориальных признаков с количеством значений меньше 50')

#### В третьем - не буду ничего предпринимать, посмотрим как поведет себя LightGBM на сырых категориальных данных

### Объединяем

In [17]:
def assemble(percentage, strategy, num_of_cat_preprocessing=3):
    data, hold_out = filter_attr(percentage)
    # ------------------------------------------
    X_num, X_cat = type_processing(data)
    X_num_pro = num_processing(X_num, strategy)
    X_cat_pro = X_cat.astype('category')
    if (num_of_cat_preprocessing==1):
        X_cat_pro = cat_1(X_cat)
    elif (num_of_cat_preprocessing==2):
        X_cat_pro = cat_2(X_cat)
    data_pro = pd.concat([X_num_pro, X_cat_pro], axis=1)
    
    # hold out тоже сразу предобработаем -------
    X_num, X_cat = type_processing(hold_out)
    X_num_pro = num_processing(X_num, strategy)
    X_cat_pro = X_cat.astype('category')
    if (num_of_cat_preprocessing==1):
        X_cat_pro = cat_1(X_cat)
    elif (num_of_cat_preprocessing==2):
        X_cat_pro = cat_2(X_cat)
    hold_out_pro = pd.concat([X_num_pro, X_cat_pro], axis=1)
    
    return data_pro, data.y, hold_out_pro, hold_out.y

# ------------------------------------------------------------------------------------

### <font color='green'>На обучении буду рассматривать Logistic Regression, XGBoost и LightGBM</font>

In [272]:
def test(estimator, param_grid, percentage, fit_params=None, strategy='mean',
         num_of_cat_preprocessing=3):
    train_X, train_y, test_X, test_y = assemble(percentage, 
                                                          strategy, num_of_cat_preprocessing)
    gscv = GridSearchCV(estimator,
             param_grid=param_grid, fit_params=fit_params,
             cv=StratifiedShuffleSplit(n_splits=3, test_size=0.25),
             scoring='roc_auc',
             verbose=False)
    gscv.fit(train_X, train_y)
    hold_out_score = np.mean(cross_val_score(gscv.best_estimator_, test_X, test_y,
            scoring='roc_auc', cv = StratifiedShuffleSplit(n_splits=3, test_size=0.25)))
    print('На данных с менее {}% Nans'.format(percentage))
    print('Score on train dataset: {}'.format(gscv.best_score_))
    print('Score on hold_out dataset: {}'.format(hold_out_score))
    print(gscv.best_params_)

### Logistic Regression

In [266]:
per_rate = [75,90,98,100]

In [267]:
param_grid_1 = {'C': [0.1,1]}

In [273]:
for percentage in per_rate:
    test(LogisticRegression(), param_grid_1, percentage, None, 'mean', 1)

На данных с менее 75% Nans
Score on train dataset: 0.6738146115975087
Score on hold_out dataset: 0.6939655328699524
{'C': 0.1}
На данных с менее 90% Nans
Score on train dataset: 0.6882505385678918
Score on hold_out dataset: 0.6421330993465048
{'C': 0.1}
На данных с менее 98% Nans
Score on train dataset: 0.6905468142692593
Score on hold_out dataset: 0.6379380152454021
{'C': 1}
На данных с менее 100% Nans
Score on train dataset: 0.6636273847824072
Score on hold_out dataset: 0.694760798020781
{'C': 1}


### XGBoost

In [173]:
param_grid_2 = {
    'max_depth': [5],
    'subsample': [0.7],
    'colsample_bytree': [0.8],
    'n_estimators': [500],
    'reg_alpha': [0.02]}

In [275]:
for percentage in per_rate:
    test(XGBClassifier(), param_grid_2, percentage, None, 'mean', 1)

На данных с менее 75% Nans
Score on train dataset: 0.7411626257525292
Score on hold_out dataset: 0.7053439884360229
{'colsample_bytree': 0.7, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 0.01, 'subsample': 0.8}
На данных с менее 90% Nans
Score on train dataset: 0.7401636414187551
Score on hold_out dataset: 0.7160051341738005
{'colsample_bytree': 0.7, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 0.01, 'subsample': 0.8}
На данных с менее 98% Nans
Score on train dataset: 0.7407386938570162
Score on hold_out dataset: 0.7042501967012208
{'colsample_bytree': 0.7, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 0.01, 'subsample': 0.8}
На данных с менее 100% Nans
Score on train dataset: 0.7433507150328058
Score on hold_out dataset: 0.7013779358639202
{'colsample_bytree': 0.7, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 0.01, 'subsample': 0.8}


<font color='green'>Посмотрим как классификатор отреагирует на появление LabelEncoder признаков</font>

In [277]:
warnings.simplefilter('ignore')
for percentage in per_rate:
    test(XGBClassifier(), param_grid_2, percentage, None, 'mean', 2)

На данных с менее 75% Nans
Score on train dataset: 0.7394604296816633
Score on hold_out dataset: 0.7144351502362228
{'colsample_bytree': 0.7, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 0.01, 'subsample': 0.8}
На данных с менее 90% Nans
Score on train dataset: 0.7402768210592813
Score on hold_out dataset: 0.698949839073625
{'colsample_bytree': 0.7, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 0.01, 'subsample': 0.8}
На данных с менее 98% Nans
Score on train dataset: 0.7359260246975224
Score on hold_out dataset: 0.7001602616398174
{'colsample_bytree': 0.7, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 0.01, 'subsample': 0.8}
На данных с менее 100% Nans
Score on train dataset: 0.7330198953315291
Score on hold_out dataset: 0.7032047493524874
{'colsample_bytree': 0.7, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 0.01, 'subsample': 0.8}


<font color='green'>Мне кажется, результат скорее положительный, еще протестим возможность заполнения Nan значений медианой вместо среднего</font>

In [278]:
for percentage in per_rate:
    test(XGBClassifier(), param_grid_2, percentage, None, 'median', 1)

На данных с менее 75% Nans
Score on train dataset: 0.7247111025368959
Score on hold_out dataset: 0.6799233016315022
{'colsample_bytree': 0.7, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 0.01, 'subsample': 0.8}
На данных с менее 90% Nans
Score on train dataset: 0.7285200589351203
Score on hold_out dataset: 0.6863059692022088
{'colsample_bytree': 0.7, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 0.01, 'subsample': 0.8}
На данных с менее 98% Nans
Score on train dataset: 0.7191496395992563
Score on hold_out dataset: 0.7284640867685042
{'colsample_bytree': 0.7, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 0.01, 'subsample': 0.8}
На данных с менее 100% Nans
Score on train dataset: 0.7284118725140293
Score on hold_out dataset: 0.7234120984243356
{'colsample_bytree': 0.7, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 0.01, 'subsample': 0.8}


### LightGBM

In [197]:
param_grid_3 ={
    'num_leaves': [15, 30],
    'reg_alpha': [0.01],
    'min_data_in_leaf': [30, 50],
    'max_depth':[3], 
    'silent':[True], 
    'metric':['auc'],                        
    'n_estimators':[100],
    'colsample_bytree':[0.8],
    'subsample':[0.7],
    'learning_rate':[0.1]
    }
fit_params_3 = {
#             "early_stopping_rounds":10, 
            "eval_metric" : 'auc', 
#             "eval_set" : [(test_X,test_y)],
            'eval_names': ['valid'],
            'verbose': 100,
            'feature_name': 'auto'
            }

<font color='green'>Сразу попробую использовать LabelEncoder</font>

In [279]:
warnings.simplefilter('ignore')
for percentage in per_rate:
    test(LGBMClassifier(), param_grid_3, percentage, fit_params_3, 'mean', 2)

На данных с менее 75% Nans
Score on train dataset: 0.7245523786827358
Score on hold_out dataset: 0.7388436264574322
{'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'metric': 'auc', 'min_data_in_leaf': 50, 'n_estimators': 100, 'num_leaves': 15, 'reg_alpha': 0.01, 'silent': True, 'subsample': 0.7}
На данных с менее 90% Nans
Score on train dataset: 0.7371766748563032
Score on hold_out dataset: 0.7018541280666959
{'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'metric': 'auc', 'min_data_in_leaf': 30, 'n_estimators': 100, 'num_leaves': 15, 'reg_alpha': 0.01, 'silent': True, 'subsample': 0.7}
На данных с менее 98% Nans
Score on train dataset: 0.7479362683628481
Score on hold_out dataset: 0.6823489812024941
{'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'metric': 'auc', 'min_data_in_leaf': 30, 'n_estimators': 100, 'num_leaves': 15, 'reg_alpha': 0.01, 'silent': True, 'subsample': 0.7}
На данных с менее 100% Nans
Score on train dataset: 0.7362548

<font color='green'>Ну и на последок посмотрим как LightGBM справится с категориальными данными</font>

In [ ]:
for percentage in per_rate:
    test(LGBMClassifier(), param_grid_3, percentage, fit_params_3, 'mean', 3)

<font color='green'>Подытоживая считаю XGBoost очевидным победителем здесь, LightGBM разочаровал на сырых данных, хотя возможно я делаю что-то не так, по идее он должен определять категориальные признаки самостоятельно. Насчет процента Nan значений, с которым работает модель, вижу закономерность в том, что чем большее данных мы отсеиваем тем ближе показатели метрик на train и hold_out выборках.</font>

In [89]:
test = pd.read_csv(r'C:\Users\preductor\Documents\MachineLearning\6_course\Сhurn_prediction\week_4\orange_small_churn_test_data.csv',
                  engine='python')

In [163]:
def final_result(test_X, estimator, param_grid, percentage,
                 fit_params=None, strategy='mean', num_of_cat_preprocessing=3):
    
    
    data = raw_data.loc[:,table_of_nan[table_of_nan.nans<percentage].index]
    data['y'] = data.labels
    data.drop(columns='labels', inplace=True)
    train_y = data.y
 
    X_num_1, X_cat_1 = type_processing(data)
    X_num_pro_1 = num_processing(X_num_1, strategy)
    
    data = test_X.loc[:,table_of_nan[table_of_nan.nans<percentage].index]
    data['y'] = np.nan
    data.drop(columns='labels', inplace=True)

    X_num_2, X_cat_2 = type_processing(data)
    X_num_pro_2 = num_processing(X_num_2, strategy)

    #-------------------------------------
    X_cat = pd.concat([X_cat_1, X_cat_2], axis=0)
    X_cat_pro = X_cat.astype('category')
    if (num_of_cat_preprocessing==1):
        X_cat_pro = cat_1(X_cat)
    elif (num_of_cat_preprocessing==2):
        X_cat_pro = cat_2(X_cat)
        
    #---------------------------------------------
    data_train = pd.concat([X_num_pro_1, X_cat_pro.iloc[:40000,:]], axis=1)    
   
    #----------------------------------------
    data_test = pd.concat([X_num_pro_2, X_cat_pro.iloc[-10000:,:]], axis=1)
    # -------------------------------------------------
    
    gscv = GridSearchCV(estimator,
             param_grid=param_grid, fit_params=fit_params,
             cv=StratifiedShuffleSplit(n_splits=3, test_size=0.25),
             scoring='roc_auc',
             verbose=False)
    gscv.fit(data_train, train_y)
    
  
    print(gscv.best_score_)
    #-------------------------------------------------
    return  data_train, train_y, data_test, gscv.best_estimator_

In [251]:
data_train, train_y, data_test, clf= final_result(test, LGBMClassifier(), 
                                param_grid_3, 95, fit_params_3, 'mean', 2)

0.7362583564898715


In [252]:
clf.fit(data_train, train_y)
prediction = clf.predict(data_test)
pd.Series(prediction).value_counts()

-1    9999
 1       1
dtype: int64

In [253]:
ans = pd.DataFrame(index=range(len(test.ID)), columns=['ID','result'])
ans['result'] = clf.predict_proba(data_test).T[1]
ans['ID'] = test.ID
ans.head(3)

,ID,result
0,0,0.076600
1,1,0.105210
2,2,0.026722


In [254]:
ans.to_csv(r'C:\Users\preductor\Documents\MachineLearning\6_course\Сhurn_prediction\week_4\submission.csv',
  sep=',', index=False)

<font color='green'>Среди перебранных мной вариантов победил LightGBM на данных с менее 100% Nan значений, OneHotEncoding для кат. признаков с менее 10 уникальными значениями и LabelEncoding для признаков с менее 50 уникальных значений. Вообще отсечение признаков с малым количеством записей по сути не принесло пользы. На Kaggle соревнование закончилось, поэтому делаю принтскрин просто своих результатов без leaderbord</font>